<a href="https://colab.research.google.com/github/22ananya/MUSI6201/blob/main/Final%20Project%20-%20Stereo%20Demixing/Run_SDR_Eval_on_Saved_Mixes_OU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import generally required packages - update as needed

In [1]:
# Import dependencies
import numpy as np
import matplotlib.pyplot as plt
import librosa
import scipy.signal as sp
import scipy.io.wavfile as wav

Import/Install the prerequisite code for implementing the Cadenza challenge - includes baselines, other important files

In [2]:
!pip install pyclarity==0.4.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 kB 994.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 1.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.7/74.7 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 2.0 MB/s eta 0:00:00
  Created wheel for antlr4-python3-runtime: filename=an

Import the dataset for the Cadenza challenge directly through the Google Drive link - Only needs to be done once! So now commented out


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Check current path

In [4]:
import os
print(os.getcwd())

/content


Change current path to Cadenza folder

In [5]:
os.chdir('/content/drive/MyDrive/Cadenza_Challenge/cad_icassp_2024')
print(os.getcwd())

/content/drive/MyDrive/Cadenza_Challenge/cad_icassp_2024


#Process single audio file through the entire model step by step

Import all dependencies - same as enhance.py file

In [6]:
from __future__ import annotations

import json
import logging
from pathlib import Path

# pylint: disable=import-error
import hydra
import numpy as np
import torch
from numpy import ndarray
from omegaconf import DictConfig
from torchaudio.pipelines import HDEMUCS_HIGH_MUSDB

from clarity.enhancer.compressor import Compressor
from clarity.enhancer.nalr import NALR
from clarity.evaluator.haaqi import compute_haaqi
from clarity.utils.audiogram import Audiogram, Listener
from clarity.utils.file_io import read_signal
from clarity.utils.flac_encoder import FlacEncoder
from clarity.utils.results_support import ResultsFile
from clarity.utils.signal_processing import (
    clip_signal,
    denormalize_signals,
    normalize_signal,
    resample,
    to_16bit,
    compute_rms,
    resample,
)
from clarity.utils.source_separation_support import get_device, separate_sources
from recipes.cad_icassp_2024.baseline.evaluate import (
    apply_gains,
    apply_ha,
    make_scene_listener_list,
    remix_stems,
    load_reference_stems,
)

logger = logging.getLogger(__name__)

/usr/local/lib/python3.10/dist-packages/recipes/cad_icassp_2024/baseline/evaluate.py:190: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path="", config_name="config")


Import all the required functions defined in Enhance.py that do not need to be changed

In [7]:
from recipes.cad_icassp_2024.baseline.enhance import (
    save_flac_signal,
    decompose_signal,
    process_remix_for_listener
)

/usr/local/lib/python3.10/dist-packages/recipes/cad_icassp_2024/baseline/enhance.py:182: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  @hydra.main(config_path="", config_name="config")


Import the correct config file (hardcoded location)

In [8]:
from omegaconf import OmegaConf
config = OmegaConf.load('baseline/config.yaml')
print(config.separator.model)

demucs


Set input directory

In [9]:
reference_folder = Path("ref_signals_100")
enhanced_mix_folder = Path("enhanced_signals_openunmix_100")

# load the audio files in provided path

In [10]:
enhanced_files = [] # initialize list of enhanced audio files
reference_files = [] # initialize list of reference files
for file in os.listdir(enhanced_mix_folder): # iterate over all files in the directory
    if file.endswith('.flac'): # if the file is an audio file
        enhanced_files.append(os.path.join(enhanced_mix_folder, file)) # add the file to the list of audio files
        reference_files.append(os.path.join(reference_folder, file.replace('.flac','_ref.flac'))) # add the corresponding ref file to the list of ref files

In [11]:
print(enhanced_files[0])
print(reference_files[0])
print(len(enhanced_files))

enhanced_signals_openunmix_100/scene_10001_L0057_remix.flac
ref_signals_100/scene_10001_L0057_remix_ref.flac
100


Load listener data

In [12]:
config.path.root = '/content/drive/MyDrive/Cadenza_Challenge/cad_icassp_2024'


In [13]:
# Load listener audiograms and songs
listener_dict = Listener.load_listener_dict(config.path.listeners_file)

with Path(config.path.gains_file).open("r", encoding="utf-8") as file:
    gains = json.load(file)

with Path(config.path.scenes_file).open("r", encoding="utf-8") as file:
    scenes = json.load(file)

with Path(config.path.scene_listeners_file).open("r", encoding="utf-8") as file:
    scenes_listeners = json.load(file)

with Path(config.path.music_file).open("r", encoding="utf-8") as file:
    songs = json.load(file)

In [ ]:
#print(songs)

Create a list of songs, listeners (audiogram) and head position (hrtf) to generate, or evaluate the mix - based on provided data

In [14]:
# Select a batch to process
scene_listener_pairs = make_scene_listener_list(
    scenes_listeners, config.evaluate.small_test
)

scene_listener_pairs = scene_listener_pairs[
    config.evaluate.batch :: config.evaluate.batch_size
]

In [15]:
print(scene_listener_pairs[0])

('scene_10001', 'L0066')


# Load functions required for audio evaluation

Add functions and variables needed for evaluation and scoring

In [16]:
scores_headers = [
    "scene",
    "song",
    "listener",
    "left_score",
    "right_score",
    "score",
]


results_file = ResultsFile(
            "scores_openunmix_SDR.csv",
            header_columns=scores_headers,
)

# Process Audio - Currently set to process a fixed number of runs (listener - scene pairings, can be changed to run entire dataset)

In [17]:
#!pip install mir_eval
!pip install museval
from museval import evaluate


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.5/963.5 kB 10.3 MB/s eta 0:00:00


# Implementing classic SDR eval

In [18]:
previous_song = ""
num_tracks = len(enhanced_files)

for idx in range(100):
    # Extract track, listener, scene info from file name
    file_name = enhanced_files[idx]
    sname = file_name.split("/", 1)[1].split("_L",1)[0]
    lname = "L" +  file_name.split("/", 1)[1].split("_L",1)[1].split("_")[0]

    # Get the listener's audiogram
    listener = listener_dict[lname]
    scene = scenes[sname]
    song_name = f"{scene['music']}-{scene['head_loudspeaker_positions']}"

    print(sname)
    print(lname)

    # load audio files for HAAQI
    reference_mixture = read_signal(
            filename=reference_files[idx],
            sample_rate=config.sample_rate,
            allow_resample=True,
        )

    enhanced_mixture = read_signal(
            filename=enhanced_files[idx],
            sample_rate=config.sample_rate,
            allow_resample=True,
        )

    # Evaluate - compare with the generated mixes
    # Compute the scores
    if np.argmax([max(np.shape(reference_mixture)), max(np.shape(enhanced_mixture))]):
        enhanced_mixture = enhanced_mixture[:max(np.shape(reference_mixture)), :]
    else:
        reference_mixture = reference_mixture[:max(np.shape(enhanced_mixture)), :]

    bss_score = evaluate(np.swapaxes(reference_mixture, 0, 1), np.swapaxes(enhanced_mixture,0,1))
    left_score = bss_score[0][0,:].mean() # SDR
    right_score = bss_score[0][1,:].mean() # SDR
    #score = np.mean([left_score, right_score])
    print(np.mean([left_score, right_score]))

        # Save scores
    results_file.add_result(
        {
            "scene": sname,
            "song": song_name,
            "listener": listener.id,
            "left_score": left_score,
            "right_score": right_score,
            "score": float(np.mean([left_score, right_score])),
        }
    )


scene_10001
L0057
-8.781786418997267
scene_10002
L0099
-13.476547769404306
scene_10002
L0051
-13.442104810118227
scene_10003
L0053
-10.959212562028195
scene_10003
L0010
-14.453750010024157
scene_10004
L0032
-13.018686549203432
scene_10004
L0078
-10.498767637341524
scene_10005
L0022
-9.869257978587047
scene_10005
L0047
-11.725056479611432
scene_10006
L0003
-11.75925775543736
scene_10006
L0095
-14.64354596398966
scene_10007
L0074
-9.665099041512542
scene_10007
L0056
-12.954331079737642
scene_10008
L0061
-8.155722299229797
scene_10008
L0076
-12.088219841378596
scene_10009
L0072
-15.050392285773542
scene_10009
L0099
-15.302060552989655
scene_10010
L0027
-14.575883994601122
scene_10010
L0072
-15.253886182606749
scene_10011
L0012
-14.533424483230647
scene_10011
L0024
-13.059745749381772
scene_10012
L0093
-10.576109918142791
scene_10012
L0041
-13.706690138447057
scene_10013
L0002
nan
scene_10013
L0008
nan
scene_10014
L0072
nan
scene_10014
L0100
nan
scene_10015
L0094
nan
scene_10015
L0017
nan


LibsndfileError: ignored

In [41]:
score = np.mean([left_score, right_score])

In [19]:
len(reference_mixture)

7560512